# Homework 2: Linear Regression
**DS4400 - Machine Learning 1**  
**Vignan Kamarthi**  
**Due: 2/13/2026 at 11pm**

## Setup

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

## Data Loading and Preprocessing

- Drop columns: `id`, `date`, `zipcode`
- Scale features to mean 0 and std 1
- Divide price by 1000

In [10]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Drop excluded columns
drop_cols = ['id', 'date', 'zipcode']
train_df = train_df.drop(columns=[c for c in drop_cols if c in train_df.columns])
test_df = test_df.drop(columns=[c for c in drop_cols if c in test_df.columns])

# Divide price by 1000
train_df['price'] = train_df['price'] / 1000
test_df['price'] = test_df['price'] / 1000

# Separate features and target
target_col = 'price'
feature_cols = [c for c in train_df.columns if c != target_col]

X_train_raw = train_df[feature_cols].values
y_train = train_df[target_col].values
X_test_raw = test_df[feature_cols].values
y_test = test_df[target_col].values

# Standardize features (fit on train, transform both)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_raw)
X_test = scaler.transform(X_test_raw)

print(f'Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features')
print(f'Testing set: {X_test.shape[0]} samples, {X_test.shape[1]} features')
print(f'Features: {feature_cols}')

Training set: 1000 samples, 18 features
Testing set: 1000 samples, 18 features
Features: ['Unnamed: 0', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


---
## Problem 2: Linear Regression with Package

### Part 1: Train and report coefficients

In [11]:
# Problem 2, Part 1: Train sklearn LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Report coefficients
coeff_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': lr_model.coef_
})
print("Intercept:", lr_model.intercept_)
print("\nCoefficients:")
print(coeff_df.to_string(index=False))

# Training metrics
y_train_pred = lr_model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print(f"\nTraining MSE: {train_mse:.4f}")
print(f"Training R^2: {train_r2:.4f}")

Intercept: 520.414834000001

Coefficients:
      Feature  Coefficient
   Unnamed: 0     8.456024
     bedrooms   -12.807339
    bathrooms    18.456913
  sqft_living    57.161582
     sqft_lot    11.127338
       floors     8.151038
   waterfront    64.230911
         view    47.610288
    condition    12.647609
        grade    92.511076
   sqft_above    48.439051
sqft_basement    27.688812
     yr_built   -68.043173
 yr_renovated    17.341926
          lat    78.129852
         long    -1.437669
sqft_living15    45.479128
   sqft_lot15   -12.906560

Training MSE: 31415.7479
Training R^2: 0.7271


### Part 2: Evaluate on testing set

In [12]:
# Problem 2, Part 2: Evaluate on testing set
y_test_pred = lr_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Testing MSE:  {test_mse:.4f}")
print(f"Testing R^2:  {test_r2:.4f}")

# Summary comparison
print("\n--- Summary ---")
print(f"{'Metric':<12} {'Train':>10} {'Test':>10}")
print(f"{'MSE':<12} {train_mse:>10.4f} {test_mse:>10.4f}")
print(f"{'R^2':<12} {train_r2:>10.4f} {test_r2:>10.4f}")

Testing MSE:  58834.6740
Testing R^2:  0.6471

--- Summary ---
Metric            Train       Test
MSE          31415.7479 58834.6740
R^2              0.7271     0.6471


### Part 3: Interpretation

On LaTeX

---
## Problem 3: Closed-Form Solution

### Implementation

Closed-form: $\theta = (X^T X)^{-1} X^T y$

Note: Must prepend a column of ones to X for the intercept term.

In [ ]:
def add_intercept(X):
    """Prepend a column of ones for the intercept term."""
    N = X.shape[0]
    return np.column_stack([np.ones(N), X])


def fit_closed_form(X, y):
    """Closed-form solution derived in class, generalized to multiple features:
    theta = (X^T X)^{-1} X^T y"""
    theta = np.linalg.inv(X.T @ X) @ X.T @ y
    return theta


def predict(X, theta):
    """Predict y given X and theta."""
    return X @ theta

In [25]:
# Add intercept column to train and test
X_train_cf = add_intercept(X_train)
X_test_cf = add_intercept(X_test)

# Fit closed-form model
theta_cf = fit_closed_form(X_train_cf, y_train)

# Report coefficients vertically
print(f"Closed-form intercept: {theta_cf[0]:.6f}\n")
print("Closed-form coefficients:")
cf_coeff_df = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': theta_cf[1:]
})
print(cf_coeff_df.to_string(index=False))

# Training metrics
y_train_pred_cf = predict(X_train_cf, theta_cf)
cf_train_mse = mean_squared_error(y_train, y_train_pred_cf)
cf_train_r2 = r2_score(y_train, y_train_pred_cf)

# Testing metrics
y_test_pred_cf = predict(X_test_cf, theta_cf)
cf_test_mse = mean_squared_error(y_test, y_test_pred_cf)
cf_test_r2 = r2_score(y_test, y_test_pred_cf)

print(f"\n--- Closed-Form Results ---")
print(f"{'Metric':<12} {'Train':>10} {'Test':>10}")
print(f"{'MSE':<12} {cf_train_mse:>10.4f} {cf_test_mse:>10.4f}")
print(f"{'R^2':<12} {cf_train_r2:>10.4f} {cf_test_r2:>10.4f}")

# Comparison with Problem 2
print(f"\n--- Comparison: sklearn vs Closed-Form ---")
print(f"{'Metric':<12} {'sklearn Train':>14} {'CF Train':>10} {'sklearn Test':>14} {'CF Test':>10}")
print(f"{'MSE':<12} {train_mse:>14.4f} {cf_train_mse:>10.4f} {test_mse:>14.4f} {cf_test_mse:>10.4f}")
print(f"{'R^2':<12} {train_r2:>14.4f} {cf_train_r2:>10.4f} {test_r2:>14.4f} {cf_test_r2:>10.4f}")

# Check coefficient match
coeff_diff = np.max(np.abs(lr_model.coef_ - theta_cf[1:]))
intercept_diff = np.abs(lr_model.intercept_ - theta_cf[0])
print(f"\nMax coefficient difference: {coeff_diff:.4f}")
print(f"Intercept difference: {intercept_diff:.4f}")

Closed-form intercept: 520.414834

Closed-form coefficients:
      Feature  Coefficient
   Unnamed: 0     8.456024
     bedrooms   -12.807339
    bathrooms    18.456913
  sqft_living    57.161582
     sqft_lot    11.127338
       floors     8.151038
   waterfront    64.230911
         view    47.610288
    condition    12.647609
        grade    92.511076
   sqft_above    48.439051
sqft_basement    27.688812
     yr_built   -68.043173
 yr_renovated    17.341926
          lat    78.129852
         long    -1.437669
sqft_living15    45.479128
   sqft_lot15   -12.906560

--- Closed-Form Results ---
Metric            Train       Test
MSE          31415.7479 58834.6740
R^2              0.7271     0.6471

--- Comparison: sklearn vs Closed-Form ---
Metric        sklearn Train   CF Train   sklearn Test    CF Test
MSE              31415.7479 31415.7479     58834.6740 58834.6740
R^2                  0.7271     0.7271         0.6471     0.6471

Max coefficient difference: 0.0000
Intercept differe

### Comparison with Problem 2

On LaTeX

---
## Problem 4: Polynomial Regression

### Implementation

For degree $p$, construct features $[X, X^2, \ldots, X^p]$, then apply closed-form from Problem 3.

In [26]:
def polynomial_features(X, degree):
    """Create polynomial features [X, X^2, ..., X^degree] from a single feature vector."""
    return np.column_stack([X**p for p in range(1, degree + 1)])

In [27]:
# Extract sqft_living feature (single feature for polynomial regression)
sqft_idx = feature_cols.index('sqft_living')
X_train_sqft_raw = X_train_raw[:, sqft_idx].reshape(-1, 1)
X_test_sqft_raw = X_test_raw[:, sqft_idx].reshape(-1, 1)

# Standardize sqft_living independently
sqft_scaler = StandardScaler()
X_train_sqft = sqft_scaler.fit_transform(X_train_sqft_raw).flatten()
X_test_sqft = sqft_scaler.transform(X_test_sqft_raw).flatten()

# Run polynomial regression for p = 1, 2, 3, 4, 5
print(f"{'p':<4} {'Train MSE':>12} {'Test MSE':>12} {'Train R^2':>12} {'Test R^2':>12}")
print("-" * 56)

for p in range(1, 6):
    # Create polynomial features
    X_train_poly = polynomial_features(X_train_sqft, p)
    X_test_poly = polynomial_features(X_test_sqft, p)
    
    # Add intercept
    X_train_poly = add_intercept(X_train_poly)
    X_test_poly = add_intercept(X_test_poly)
    
    # Fit using closed-form from Problem 3
    theta_poly = fit_closed_form(X_train_poly, y_train)
    
    # Predictions
    y_train_pred_poly = predict(X_train_poly, theta_poly)
    y_test_pred_poly = predict(X_test_poly, theta_poly)
    
    # Metrics
    poly_train_mse = mean_squared_error(y_train, y_train_pred_poly)
    poly_test_mse = mean_squared_error(y_test, y_test_pred_poly)
    poly_train_r2 = r2_score(y_train, y_train_pred_poly)
    poly_test_r2 = r2_score(y_test, y_test_pred_poly)
    
    print(f"{p:<4} {poly_train_mse:>12.4f} {poly_test_mse:>12.4f} {poly_train_r2:>12.4f} {poly_test_r2:>12.4f}")

p       Train MSE     Test MSE    Train R^2     Test R^2
--------------------------------------------------------
1      57947.5262   88575.9785       0.4967       0.4687
2      54822.6651   71791.6795       0.5238       0.5694
3      53785.1947   99833.4838       0.5329       0.4012
4      52795.7748  250979.2743       0.5415      -0.5053
5      52626.1120  570616.9148       0.5429      -2.4225


### Observations

On LaTeX

---
## Problem 5: Gradient Descent

### Part 1: Implementation

Update rule: $\theta := \theta - \alpha \cdot \frac{1}{N} X^T(X\theta - y)$

In [33]:
def gradient_descent(X, y, alpha, n_iterations, theta_init=None):
    """
    Gradient descent for linear regression.
    
    Parameters:
        X: feature matrix with intercept column (N x (d+1))
        y: target vector (N,)
        alpha: learning rate
        n_iterations: number of iterations
        theta_init: initial parameter vector (optional, defaults to zeros)
    
    Returns:
        theta: learned parameters
        loss_history: list of MSE at each iteration
    """
    N, d = X.shape
    if theta_init is None:
        theta = np.zeros(d)
    else:
        theta = theta_init.copy()
    
    loss_history = []
    
    for i in range(n_iterations):
        # Gradient: (1/N) * X^T (X*theta - y)
        residuals = X @ theta - y
        gradient = (1 / N) * (X.T @ residuals)
        
        # Update
        theta = theta - alpha * gradient
        
        # Track MSE
        mse = np.mean(residuals**2)
        loss_history.append(mse)
    
    return theta, loss_history

### Part 2: Results

In [36]:
# Problem 5, Part 2: Vary learning rate and iterations
alphas = [0.01, 0.05, 0.1]
checkpoints = [10, 50, 100]

# Use X_train_cf and X_test_cf (with intercept) from Problem 3
print(f"{'alpha':<8} {'iters':<8} {'Train MSE':>14} {'Test MSE':>14} {'Train R^2':>12} {'Test R^2':>12}")
print("-" * 75)

for alpha in alphas:
    for n_iter in checkpoints:
        theta_gd, loss_hist = gradient_descent(X_train_cf, y_train, alpha, n_iter)
        
        # Predictions
        y_train_pred_gd = predict(X_train_cf, theta_gd)
        y_test_pred_gd = predict(X_test_cf, theta_gd)
        
        # Metrics
        gd_train_mse = mean_squared_error(y_train, y_train_pred_gd)
        gd_test_mse = mean_squared_error(y_test, y_test_pred_gd)
        gd_train_r2 = r2_score(y_train, y_train_pred_gd)
        gd_test_r2 = r2_score(y_test, y_test_pred_gd)
        
        print(f"{alpha:<8} {n_iter:<8} {gd_train_mse:>14.2f} {gd_test_mse:>14.2f} {gd_train_r2:>12.4f} {gd_test_r2:>12.4f}")
    print()

# Compare best GD result with closed-form
print("--- Closed-form reference ---")
print(f"{'CF':<8} {'--':<8} {cf_train_mse:>14.2f} {cf_test_mse:>14.2f} {cf_train_r2:>12.4f} {cf_test_r2:>12.4f}")

alpha    iters         Train MSE       Test MSE    Train R^2     Test R^2
---------------------------------------------------------------------------
0.01     10            294796.69      352448.96      -1.5604      -1.1139
0.01     50            138298.52      170450.93      -0.2012      -0.0223
0.01     100            70094.38       94751.22       0.3912       0.4317

0.05     10            135793.77      167419.60      -0.1794      -0.0042
0.05     50             33384.66       58922.60       0.7100       0.6466
0.05     100            31514.36       58907.87       0.7263       0.6467

0.1      10             66473.61       90873.01       0.4227       0.4550
0.1      50             31510.72       58929.63       0.7263       0.6466
0.1      100            31427.50       58891.74       0.7270       0.6468

--- Closed-form reference ---
CF       --             31415.75       58834.67       0.7271       0.6471


### Part 3: Observations

On LaTeX

---
## Problem 6: Ridge Regularization

### Part 2: Ridge regression with gradient descent

Modified gradient: $\nabla J(\theta) = \frac{1}{N} X^T(X\theta - y) + \frac{2\lambda}{N} \theta$

Note: Typically do not regularize the intercept term $\theta_0$.

In [37]:
def gradient_descent_ridge(X, y, alpha, n_iterations, lam, theta_init=None):
    """
    Gradient descent for ridge regression.
    
    Parameters:
        X: feature matrix with intercept column (N x (d+1))
        y: target vector (N,)
        alpha: learning rate
        n_iterations: number of iterations
        lam: regularization parameter lambda
        theta_init: initial parameter vector (optional, defaults to zeros)
    
    Returns:
        theta: learned parameters
        loss_history: list of loss at each iteration
    """
    N, d = X.shape
    if theta_init is None:
        theta = np.zeros(d)
    else:
        theta = theta_init.copy()
    
    loss_history = []
    
    for i in range(n_iterations):
        # Gradient: (1/N) * X^T(X*theta - y) + (2*lambda/N) * theta
        residuals = X @ theta - y
        gradient = (1 / N) * (X.T @ residuals)
        
        # Add regularization term (don't regularize intercept theta[0])
        reg_term = (2 * lam / N) * theta
        reg_term[0] = 0  # no penalty on intercept
        gradient += reg_term
        
        # Update
        theta = theta - alpha * gradient
        
        # Track ridge loss: MSE + lambda * ||theta[1:]||^2
        mse = np.mean(residuals**2)
        ridge_loss = mse + lam * np.sum(theta[1:]**2)
        loss_history.append(ridge_loss)
    
    return theta, loss_history

### Part 3: Simulation study

Generate: $X_i \sim \text{Uniform}[-2, 2]$, $Y_i = 1 + 2X_i + e_i$, $e_i \sim N(0, 2)$

In [39]:
# Problem 6, Part 3: Simulation study
np.random.seed(42)

# Generate data: X ~ Uniform[-2, 2], Y = 1 + 2X + e, e ~ N(0, 2)
N_sim = 1000
X_sim = np.random.uniform(-2, 2, N_sim)
e_sim = np.random.normal(0, 2, N_sim)
y_sim = 1 + 2 * X_sim + e_sim

# Prepare feature matrix with intercept
X_sim_mat = add_intercept(X_sim.reshape(-1, 1))

# Settings
lambdas = [0, 1, 10, 100, 1000, 10000]
n_iter_sim = 1000

print(f"{'Lambda':<10} {'Slope':>10} {'Intercept':>12} {'MSE':>12} {'R^2':>10}")
print("-" * 58)

for lam in lambdas:
    # Adaptive learning rate: large lambda makes the gradient huge,
    # so we need a smaller step size to avoid divergence.
    # The dominant eigenvalue of the Hessian scales with (||X^T X|| + lambda),
    # so alpha ~ 1 / (||X^T X|| + lambda) keeps things stable.
    alpha_sim = 0.1 / (1 + lam / 1000)
    
    if lam == 0:
        theta_sim, _ = gradient_descent(X_sim_mat, y_sim, alpha_sim, n_iter_sim)
    else:
        theta_sim, _ = gradient_descent_ridge(X_sim_mat, y_sim, alpha_sim, n_iter_sim, lam)
    
    y_sim_pred = predict(X_sim_mat, theta_sim)
    sim_mse = mean_squared_error(y_sim, y_sim_pred)
    sim_r2 = r2_score(y_sim, y_sim_pred)
    
    label = "None" if lam == 0 else str(lam)
    print(f"{label:<10} {theta_sim[1]:>10.4f} {theta_sim[0]:>12.4f} {sim_mse:>12.4f} {sim_r2:>10.4f}")

print(f"\nTrue parameters: intercept = 1, slope = 2")

Lambda          Slope    Intercept          MSE        R^2
----------------------------------------------------------
None           1.9226       1.1948       3.8999     0.5639
1              1.9198       1.1947       3.8999     0.5639
10             1.8948       1.1937       3.9009     0.5638
100            1.6768       1.1852       3.9823     0.5547
1000           0.7796       1.1502       5.6821     0.3646
10000          0.1228       1.1245       8.3189     0.0697

True parameters: intercept = 1, slope = 2


### Observations

On LaTeX